# Keras tuner

In [ ]:
!pip install -q -U keras-tuner
import keras_tuner as kt
import time
from pathlib import Path

In [ ]:
LOG_DIR = Path() / 'kerastuner_logs' / str(int(time.time()))
LOG_DIR.mkdir(parents=True, exist_ok=True)

The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round

- https://www.tensorflow.org/tutorials/keras/keras_tuner
- https://arxiv.org/pdf/1603.06560.pdf

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/AN2DL/Homework1

## Import libraries

In [ ]:
import random
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
plt.style.use('ggplot')

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from keras import layers

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

# Global settings

In [ ]:
IMAGE_SHAPE = [96,96]
INPUT_SHAPE = (*IMAGE_SHAPE,3)
BATCH_SIZE = 16
SEED = 42
DATASET_DIR = Path() / 'training_data_final'
MODELS_DIR = Path() / 'models'
MODELS_DIR.mkdir(parents=True, exist_ok=True)
CLASSES = [f'Species{i+1}' for i in range(8)]
NUM_CLASSES = len(CLASSES)

In [ ]:
tf.random.set_seed(SEED)

# Instantiate the dataset generators

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train generator with augmentation
train_image_gen  = ImageDataGenerator(rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      zoom_range=[0.5,1.5],
                                      brightness_range=[0.5,1.5],
                                      shear_range=0.2,
                                      vertical_flip=True,
                                      horizontal_flip=True,
                                      fill_mode='reflect',
                                      validation_split = 0.15,
                                      )

# validation generator without augmentation
validation_image_gen = ImageDataGenerator(validation_split = 0.15)

train_dataset = train_image_gen.flow_from_directory(directory=DATASET_DIR,
                                                    target_size=IMAGE_SHAPE,
                                                    color_mode='rgb',
                                                    classes=None,
                                                    class_mode='categorical',
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=SEED,
                                                    subset='training',
                                                    )

validation_dataset = validation_image_gen.flow_from_directory(directory=DATASET_DIR,
                                                              target_size=IMAGE_SHAPE,
                                                              color_mode='rgb',
                                                              classes=None,
                                                              class_mode='categorical',
                                                              batch_size=BATCH_SIZE,
                                                              shuffle=False,
                                                              seed=SEED,
                                                              subset='validation'
                                                              )

## Tuning VGG16

In [ ]:
conv_base  = keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224,3)
)
conv_base.trainable = False

In [ ]:
def model_builder(hp):

    inputs = keras.Input(shape=INPUT_SHAPE)

    x = layers.Resizing(224, 224, interpolation='bicubic')(inputs)
    x = keras.applications.vgg16.preprocess_input(x)
    x = conv_base(x)

    x = layers.GlobalAveragePooling2D()(x)
    
    # best activation
    hp_activation = hp.Choice('activation', values=['relu','tanh'])
    
    # best number of layers
    for i in range(hp.Int('n_layers',1,4)):
        # best number of neurons
        x = layers.Dense(units=hp.Int(f'units_{i}', min_value=128, max_value=512, step=128))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(hp_activation)(x)
        x = layers.Dropout(0.3)(x)


    outputs = layers.Dense(NUM_CLASSES, activation='softmax', kernel_initializer=keras.initializers.GlorotUniform(seed=SEED))(x)

    model = keras.Model(inputs, outputs)

    # best lr
    # hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])

    model.compile(loss=keras.losses.CategoricalCrossentropy(),
                  optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

    return model

In [ ]:
tuner_vgg16 = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory=LOG_DIR,
    project_name='VGG16')

tuner_vgg16.search(train_dataset,
             epochs=50,
             validation_data=validation_dataset,
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

In [ ]:
tuner_vgg16.search_space_summary()

Get the optimal hyperparameters.

In [ ]:
best_hp = tuner_vgg16.get_best_hyperparameters()[0]
best_hp.values
#tuner_vgg16.get_best_models()[0].summary()

Summary of the model with the optimal hyperparameters.

In [ ]:
model = tuner_vgg16.hypermodel.build(best_hp)
model.summary()

## Tuning ResNet-50

In [ ]:
conv_base  = keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224,3)
)
conv_base.trainable = False

In [ ]:
def model_builder(hp):

    inputs = keras.Input(shape=INPUT_SHAPE)

    x = layers.Resizing(224, 224, interpolation='bicubic')(inputs)
    x = keras.applications.resnet.preprocess_input(x)
    x = conv_base(x)

    x = layers.GlobalAveragePooling2D()(x)
    
    # best activation
    hp_activation = hp.Choice('activation', values=['relu','tanh'])
    
    # best number of layers
    for i in range(hp.Int('n_layers',1,4)):
        # best number of neurons
        x = layers.Dense(units=hp.Int(f'units_{i}', min_value=128, max_value=512, step=128))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(hp_activation)(x)
        x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(NUM_CLASSES, activation='softmax', kernel_initializer=keras.initializers.GlorotUniform(seed=SEED))(x)

    model = keras.Model(inputs, outputs)

    # best lr
    # hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])

    model.compile(loss=keras.losses.CategoricalCrossentropy(),
                  optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

    return model

In [ ]:
tuner_resnet50 = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory=LOG_DIR,
    project_name='ResNet-50')

tuner_resnet50.search(train_dataset,
             epochs=50,
             validation_data=validation_dataset,
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

In [ ]:
tuner_resnet50.search_space_summary()

Get the optimal hyperparameters.

In [ ]:
best_hp = tuner_resnet50.get_best_hyperparameters()[0]
best_hp.values
#tuner_resnet50.get_best_models()[0].summary()

Summary of the model with the optimal hyperparameters.

In [ ]:
model = tuner_resnet50.hypermodel.build(best_hp)
model.summary()